# 01. Importing libraries

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# project folder path
path = r'/Users/achestnut/Desktop/Instacart Data Analysis'

# 02. Importing data

In [3]:
# import orders_prods_merged2
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged2.pkl'))

In [4]:
# create a subset of first one million entries
df_ords_prods_merge = ords_prods_merge[:1000000]

In [5]:
df_ords_prods_merge.shape

(1000000, 19)

# 03. Finding aggregated mean

In [6]:
# aggregated mean for subset of dataframe
df_ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

order_number
                      mean
department_id             
4                18.825780
7                17.472355
13               17.993423
14               19.246334
16               19.463012
17               11.294069
19               19.305237
20               17.599636

In [7]:
# aggregated mean for entire dataframe
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

order_number
                      mean
department_id             
1                15.457838
2                17.277920
3                17.170395
4                17.811403
5                15.215751
6                16.439806
7                17.225802
8                15.340650
9                15.895474
10               20.197148
11               16.170638
12               15.887671
13               16.583536
14               16.773669
15               16.165037
16               17.665606
17               15.694469
18               19.310397
19               17.177343
20               16.473447
21               22.902379

After finding the aggregated mean of the 'order_number' column grouped by 'department_id' of both the subset and the entire dataframe, the results show that the aggregated mean value of the subset is greater than the aggregated mean value of the entire dataframe. 

# 04. Creating a flag

In [8]:
# create max order column
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [9]:
ords_prods_merge.head(15)

order_id  user_id eval_set  order_number  orders_day_of_week  \
0    2539329        1    prior             1                   2   
1    2398795        1    prior             2                   3   
2     473747        1    prior             3                   3   
3    2254736        1    prior             4                   4   
4     431534        1    prior             5                   4   
5    3367565        1    prior             6                   2   
6     550135        1    prior             7                   1   
7    3108588        1    prior             8                   1   
8    2295261        1    prior             9                   1   
9    2550362        1    prior            10                   4   
10   2968173       15    prior            15                   1   
11   1870022       15    prior            17                   2   
12   1911383       15    prior            18                   2   
13   2715276       15    prior            21                   1   
14    487368       15    prior            22                   1   

    hour_of_order  days_since_prior_order  product_id  add_to_cart_order  \
0               8                     0.0         196                  1   
1               7                    15.0         196                  1   
2              12                    21.0         196                  1   
3               7                    29.0         196                  1   
4              15                    28.0         196                  1   
5               7                    19.0         196                  1   
6               9                    20.0         196                  1   
7              14                    14.0         196                  2   
8              16                     0.0         196                  4   
9               8                    30.0         196                  1   
10              9                     7.0         196                  2   
11             16                     8.0         196                  6   
12             11                     7.0         196                  1   
13              9                     7.0         196                  1   
14             10                    14.0         196                  1   

    reordered product_name  aisle_id  department_id  prices _merge  \
0           0         Soda        77              7     9.0   both   
1           1         Soda        77              7     9.0   both   
2           1         Soda        77              7     9.0   both   
3           1         Soda        77              7     9.0   both   
4           1         Soda        77              7     9.0   both   
5           1         Soda        77              7     9.0   both   
6           1         Soda        77              7     9.0   both   
7           1         Soda        77              7     9.0   both   
8           1         Soda        77              7     9.0   both   
9           1         Soda        77              7     9.0   both   
10          0         Soda        77              7     9.0   both   
11          1         Soda        77              7     9.0   both   
12          1         Soda        77              7     9.0   both   
13          1         Soda        77              7     9.0   both   
14          1         Soda        77              7     9.0   both   

       busiest_day    price_range_loc     busiest_days busiest_period_of_day  \
0   Regularly busy  Mid range product   Regularly busy        Average orders   
1   Regularly busy  Mid range product  Least busy days        Average orders   
2   Regularly busy  Mid range product  Least busy days           Most orders   
3       Least busy  Mid range product  Least busy days        Average orders   
4       Least busy  Mid range product  Least busy days           Most orders   
5   Regularly busy  Mid range product   Regularly busy        Average orders   
6   Regularly bus

In [10]:
# create loyal customer flag
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [11]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [12]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [13]:
# frequency of loyalty flag
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

# 05. Mean of product prices by loyalty flag

In [14]:
# group by 'prices'
ords_prods_merge.groupby('prices')

In [15]:
# mean of product prices for each loyalty category
ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['mean']})

prices
                       mean
loyalty_flag               
Loyal customer    10.386336
New customer      13.294670
Regular customer  12.495717

# 06. Create spending flag

In [16]:
# group data by 'user_id' column and generate mean product price
ords_prods_merge['mean_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [17]:
# check group
ords_prods_merge.head(15)

order_id  user_id eval_set  order_number  orders_day_of_week  \
0    2539329        1    prior             1                   2   
1    2398795        1    prior             2                   3   
2     473747        1    prior             3                   3   
3    2254736        1    prior             4                   4   
4     431534        1    prior             5                   4   
5    3367565        1    prior             6                   2   
6     550135        1    prior             7                   1   
7    3108588        1    prior             8                   1   
8    2295261        1    prior             9                   1   
9    2550362        1    prior            10                   4   
10   2968173       15    prior            15                   1   
11   1870022       15    prior            17                   2   
12   1911383       15    prior            18                   2   
13   2715276       15    prior            21                   1   
14    487368       15    prior            22                   1   

    hour_of_order  days_since_prior_order  product_id  add_to_cart_order  \
0               8                     0.0         196                  1   
1               7                    15.0         196                  1   
2              12                    21.0         196                  1   
3               7                    29.0         196                  1   
4              15                    28.0         196                  1   
5               7                    19.0         196                  1   
6               9                    20.0         196                  1   
7              14                    14.0         196                  2   
8              16                     0.0         196                  4   
9               8                    30.0         196                  1   
10              9                     7.0         196                  2   
11             16                     8.0         196                  6   
12             11                     7.0         196                  1   
13              9                     7.0         196                  1   
14             10                    14.0         196                  1   

    reordered  ... department_id  prices  _merge     busiest_day  \
0           0  ...             7     9.0    both  Regularly busy   
1           1  ...             7     9.0    both  Regularly busy   
2           1  ...             7     9.0    both  Regularly busy   
3           1  ...             7     9.0    both      Least busy   
4           1  ...             7     9.0    both      Least busy   
5           1  ...             7     9.0    both  Regularly busy   
6           1  ...             7     9.0    both  Regularly busy   
7           1  ...             7     9.0    both  Regularly busy   
8           1  ...             7     9.0    both  Regularly busy   
9           1  ...             7     9.0    both      Least busy   
10          0  ...             7     9.0    both  Regularly busy   
11          1  ...             7     9.0    both  Regularly busy   
12          1  ...             7     9.0    both  Regularly busy   
13          1  ...             7     9.0    both  Regularly busy   
14          1  ...             7     9.0    both  Regularly busy   

      price_range_loc     busiest_days busiest_period_of_day max_order  \
0   Mid range product   Regularly busy        Average orders        10   
1   Mid range product  Least busy days        Average orders        10   
2   Mid range product  Least busy days           Most orders        10   
3   Mid range product  Least busy days        Average orders        10   
4   Mid range product  Least busy days           Most orders        10   
5   Mid range product   Regularly busy        Average orders        10   
6   Mid range product     Busiest days           Most orders        10   
7   Mid range pro

In [18]:
# Create a spending flag for each user based on the average price across all their orders
ords_prods_merge.loc[ords_prods_merge['mean_price'] < 10, 'spending_flag'] = 'Low spender'

In [19]:
ords_prods_merge.loc[ords_prods_merge['mean_price'] >=10, 'spending_flag'] = 'High spender'

In [20]:
# frequency of spending flags
ords_prods_merge['spending_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spending_flag, dtype: int64

# 07. Create an order frequency flag

In [22]:
# group data by 'user_id' column and generate median days since prior order
ords_prods_merge['median_days'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [23]:
# check group
ords_prods_merge.head(15)

order_id  user_id eval_set  order_number  orders_day_of_week  \
0    2539329        1    prior             1                   2   
1    2398795        1    prior             2                   3   
2     473747        1    prior             3                   3   
3    2254736        1    prior             4                   4   
4     431534        1    prior             5                   4   
5    3367565        1    prior             6                   2   
6     550135        1    prior             7                   1   
7    3108588        1    prior             8                   1   
8    2295261        1    prior             9                   1   
9    2550362        1    prior            10                   4   
10   2968173       15    prior            15                   1   
11   1870022       15    prior            17                   2   
12   1911383       15    prior            18                   2   
13   2715276       15    prior            21                   1   
14    487368       15    prior            22                   1   

    hour_of_order  days_since_prior_order  product_id  add_to_cart_order  \
0               8                     0.0         196                  1   
1               7                    15.0         196                  1   
2              12                    21.0         196                  1   
3               7                    29.0         196                  1   
4              15                    28.0         196                  1   
5               7                    19.0         196                  1   
6               9                    20.0         196                  1   
7              14                    14.0         196                  2   
8              16                     0.0         196                  4   
9               8                    30.0         196                  1   
10              9                     7.0         196                  2   
11             16                     8.0         196                  6   
12             11                     7.0         196                  1   
13              9                     7.0         196                  1   
14             10                    14.0         196                  1   

    reordered  ... _merge     busiest_day    price_range_loc     busiest_days  \
0           0  ...   both  Regularly busy  Mid range product   Regularly busy   
1           1  ...   both  Regularly busy  Mid range product  Least busy days   
2           1  ...   both  Regularly busy  Mid range product  Least busy days   
3           1  ...   both      Least busy  Mid range product  Least busy days   
4           1  ...   both      Least busy  Mid range product  Least busy days   
5           1  ...   both  Regularly busy  Mid range product   Regularly busy   
6           1  ...   both  Regularly busy  Mid range product     Busiest days   
7           1  ...   both  Regularly busy  Mid range product     Busiest days   
8           1  ...   both  Regularly busy  Mid range product     Busiest days   
9           1  ...   both      Least busy  Mid range product  Least busy days   
10          0  ...   both  Regularly busy  Mid range product     Busiest days   
11          1  ...   both  Regularly busy  Mid range product   Regularly busy   
12          1  ...   both  Regularly busy  Mid range product   Regularly busy   
13          1  ...   both  Regularly busy  Mid range product     Busiest days   
14          1  ...   both  Regularly busy  Mid range product     Busiest days   

   busiest_period_of_day max_order      loyalty_flag mean_price spending_flag  \
0         Average orders        10      New customer   6.367797   Low spender   
1         Average orders        10      New customer   6.367797   Low spender   
2            Most orders        10      New customer   6.367797   Low spender   
3         Average orders        10      New customer   6.367797   Low spender

In [24]:
# create frquency flags
ords_prods_merge.loc[ords_prods_merge['median_days'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [25]:
ords_prods_merge.loc[(ords_prods_merge['median_days'] > 10) & (ords_prods_merge['median_days'] <= 20), 'frequency_flag'] = 'Regular customer'

In [26]:
ords_prods_merge.loc[ords_prods_merge['median_days'] <=10, 'frequency_flag'] = 'Frequent customer'

In [27]:
# frequency of order frequency flag
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

Frequent customer        22796659
Regular customer          6921472
Non-frequent customer     2686728
Name: frequency_flag, dtype: int64

# 08. Export data 

In [28]:
# export data as pickle
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged3.pkl'))